In [1]:
from azure.ai.formrecognizer import FormRecognizerClient, RecognizedForm
from azure.core.credentials import AzureKeyCredential

In [2]:
FORMRECOGNIZER_ENDPOINT = 'https://fr-usw2.cognitiveservices.azure.com'
FORMRECOGNIZER_KEY = '1be1b82af7084c85b6b9c45f6b198d7a'

IMAGE_URL = 'https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/simple-invoice.png'
IMAGE_FILE = 'simple-invoice.png'

In [3]:
credential = AzureKeyCredential(FORMRECOGNIZER_KEY)
client = FormRecognizerClient(FORMRECOGNIZER_ENDPOINT, credential)
# task = client.begin_recognize_invoices_from_url(IMAGE_URL)
with open(IMAGE_FILE, 'rb') as f:
    data = f.read()
task = client.begin_recognize_invoices(data)
print(task.status())
analyze_result = task.result()
print(analyze_result)

InProgress
[RecognizedForm(form_type=prebuilt:invoice, fields={'CustomerAddress': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=1020 Enterprise Way Sunnayvale, CA 87659, bounding_box=[Point(x=1221.3, y=399.5), Point(x=1567.3, y=401.6), Point(x=1566.8, y=480.1), Point(x=1220.8, y=478.0)], field_elements=None), name=CustomerAddress, value='1020 Enterprise Way Sunnayvale, CA 87659', confidence=0.954), 'CustomerAddressRecipient': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=Microsoft, bounding_box=[Point(x=1216.0, y=353.0), Point(x=1373.0, y=353.0), Point(x=1373.0, y=384.0), Point(x=1216.0, y=385.0)], field_elements=None), name=CustomerAddressRecipient, value='Microsoft', confidence=0.981), 'CustomerName': FormField(value_type=string, label_data=None, value_data=FieldData(page_number=1, text=Microsoft, bounding_box=[Point(x=1216.0, y=353.0), Point(x=1373.0, y=353.0), Point(x=1373.0, y=384.0), Point(x=1216.0,